# Tutorial on manipulating SYN L2 VGS composite products from the cloud

In [ ]:
import xarray as xr
from pathlib import Path
from xarray_eop.eop import open_eop_datatree

### Get AWS credentials

In [ ]:
import s3fs
import json
# Assuming credentials are in ~/.eopf/secrets.json
try:
    SECRET_PATH = Path.home() / Path(".eopf/secrets.json")
    with open(SECRET_PATH) as f:
        secrets=json.load(f)

# Or use ENV variables
except:
    import os
    secrets = {"s3input" : {
        "key": os.getenv("AWS_ACCESS_KEY_ID"),
        "secret": os.getenv("AWS_SECRET_ACCES_KEY"),
        "endpoint_url": os.getenv("AWS_ENDPOINT_URL"),
        "region_name": os.getenv("AWS_DEFAULT_REGION")
        }
    }

secrets["s3input"].pop("region_name",None)



### Browse S3 buckets and get SYNAOD product

In [ ]:
import s3fs
SAMPLE_PATH = "s3://s3-input/Products/NewFormat/"
s3 = s3fs.S3FileSystem(
    key=secrets["s3input"]["key"],
    secret=secrets["s3input"]["secret"],
    endpoint_url=secrets["s3input"]["endpoint_url"]
)
s3_path = s3.glob(SAMPLE_PATH+"S3SYNV10*.zip")
store=f"zip::s3://{s3_path[0]}"
store

### Open the whole product with datatree

In [ ]:
dt = open_eop_datatree(store,decode_times=False,backend_kwargs={"storage_options": {"s3":secrets["s3input"]}})
dt

### Opening gridded measurement data with xarray

In [ ]:
ds=dt.measurements
b0 = ds.b0
b0

Underlying data is dask.array

In [ ]:
b0.data

#### Simple raster plot

In [ ]:
b0.plot()

#### Plot using the coordinates (lon,lat)

In [ ]:
import matplotlib.pyplot as plt

b0=b0.dropna("latitude",how="all")
b0=b0.dropna("longitude",how="all")
plt.figure(figsize=(14, 6))
ax = plt.axes()
b0.plot.pcolormesh(
    ax=ax,
    x="longitude", y="latitude", add_colorbar=False
)

#### Open auxiliary data

In [ ]:
ds=dt.conditions.auxiliary

In [ ]:
var = ds.ag
var=var.dropna("latitude",how="all")
var=var.dropna("longitude",how="all")
plt.figure(figsize=(14, 6))
ax = plt.axes()
var.plot.pcolormesh(
    ax=ax,
    x="longitude", y="latitude", add_colorbar=False
)